### Data Collection

In [1]:
import pandas as pd
from os import path
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
import sklearn

# Edit path if need be (shouldn't need to b/c we all have the same folder structure)
CSV_PATH_1 = '../Videos/all_data'
CSV_PATH_2 = '../Videos2/all_data2'
FILE_EXTENSION = '_all.csv'
GENRES = ['country', 'edm', 'pop', 'rap', 'rock']

# Containers for the data frames
genre_dfs = {}
all_genres = None

# Read in the 5 genre's of CV's
for genre in GENRES:
    genre_csv_path_1 = path.join(CSV_PATH_1, genre) + FILE_EXTENSION
    genre_csv_path_2 = path.join(CSV_PATH_2, genre) + FILE_EXTENSION
    df_1 = pd.read_csv(genre_csv_path_1)
    df_2 = pd.read_csv(genre_csv_path_2)
    df_1 = df_1.drop('Unnamed: 0',1)
    df_2 = df_2.drop('Unnamed: 0',1)
    df_combined = pd.concat([df_1,df_2],ignore_index=True)
    genre_dfs[genre] = df_combined

all_genres = pd.concat(genre_dfs.values())
all_genres.head()

# genre_dfs is now a dictionary that contains the 5 different data frames
# all_genres is a dataframe that contains all of the data

,filename,author,description,viewcount,rating,likes,dislikes,duration,length,keywords,...,colors_8_red,colors_8_blue,colors_8_green,colors_9_red,colors_9_blue,colors_9_green,colors_10_red,colors_10_blue,colors_10_green,genre
0,Luke Bryan - Roller Coaster.mp4,LukeBryanVEVO,Luke Bryan - Crash My Party\nPurchase now on i...,28948653,4.840108,127866,5324,00:04:23,263,"[Luke, Bryan, Roller, Coaster, Capitol, Record...",...,230,210,190,90,70,70,240,220,200,country
1,Dierks Bentley - Drunk On A Plane.mp4,DierksBentleyVEVO,Purchase Dierks Bentley’s latest music: http:/...,41548786,4.763639,140682,8835,00:04:51,291,"[Dierks, Bentley, Drunk, On, Plane, Capitol, R...",...,70,50,50,100,110,120,90,70,70,country
2,Thomas Rhett - Get Me Some Of That.mp4,ThomasRhettVEVO,Music video by Thomas Rhett performing Get Me ...,43868160,4.826069,128488,5841,00:03:13,193,"[Thomas, Rhett, Get, Me, Some, Of, That, The, ...",...,40,50,30,50,70,50,40,60,50,country
3,David Nail - Whatever She's Got.mp4,DavidNailVEVO,Purchase David Nail’s latest music: http://umg...,48648247,4.826632,141108,6393,00:04:01,241,"[David, Nail, Whatever, She's, Got, MCA, Nashv...",...,60,50,30,60,40,30,70,60,50,country
4,Joe Nichols - Yeah.mp4,JoeNicholsVEVO,Joe Nichols - Yeah\n“Yeah” from Joe Nichol’s C...,11397694,4.815725,33255,1606,00:03:52,232,"[Joe Nichols, Red Bow Records, Country, Yeah]",...,30,50,50,20,40,60,20,30,50,country


### Ordinal Genres
Below, we make the genres ordinal to fit in the random forest classifiers. We add a new column to our dataframe to do so, write a function to populate it, and run it across the dataframe.

In [2]:
def genre_to_ordinal(genre_in):
    if(genre_in == "country"):
        return 0
    elif(genre_in == "pop"):
        return 1
    elif(genre_in == "rock"):
        return 2
    elif(genre_in == "edm"):
        return 3
    elif(genre_in == "rap"):
        return 4
    else:
        return genre_in
    
all_genres['genre_ordinal'] = all_genres.genre.apply(genre_to_ordinal)

We add in some boolean genre classifiers to make our analysis more fine-grained. Rather than saying "we predict this video is country with 50% confidence", we could say "we predict this video is not edm with 90% confidence" and so on.

In [3]:
# Adding is_country flag
def is_country(genre_in):
    if(genre_in == "country"):
        return 1
    else:
        return 0
    
all_genres['is_country'] = all_genres.genre.apply(is_country)

# Adding is_country flag
def is_rock(genre_in):
    if(genre_in == "rock"):
        return 1
    else:
        return 0
    
all_genres['is_rock'] = all_genres.genre.apply(is_rock)

# Adding is_edm flag
def is_edm(genre_in):
    if(genre_in == "edm"):
        return 1
    else:
        return 0
    
all_genres['is_edm'] = all_genres.genre.apply(is_edm)

# Adding is_rap flag
def is_rap(genre_in):
    if(genre_in == "rap"):
        return 1
    else:
        return 0
    
all_genres['is_rap'] = all_genres.genre.apply(is_rap)

# Adding is_country flag
def is_pop(genre_in):
    if(genre_in == "pop"):
        return 1
    else:
        return 0
    
all_genres['is_pop'] = all_genres.genre.apply(is_pop)

### Test and Train Sets
We create our training and test sets by splitting all_genres by genre, and making 10 of each genre train and 10 test. We aggregate by genre to make our full train and full test sets, each containing 50 records of various genres.

In [4]:
# Subset all_genres to group by individual genres
country_records  = all_genres[all_genres["genre"] == "country"]
rock_records     = all_genres[all_genres["genre"] == "rock"]
pop_records      = all_genres[all_genres["genre"] == "pop"]
edm_records      = all_genres[all_genres["genre"] == "edm"]
rap_records      = all_genres[all_genres["genre"] == "rap"]

# From the subsets above, create train and test sets from each
country_train = country_records.head(len(country_records) / 2)
country_test  = country_records.tail(len(country_records) / 2)
rock_train    = rock_records.head(len(rock_records) / 2)
rock_test     = rock_records.tail(len(rock_records) / 2)
pop_train     = pop_records.head(len(pop_records) / 2)
pop_test      = pop_records.tail(len(pop_records) / 2)
edm_train     = edm_records.head(len(edm_records) / 2)
edm_test      = edm_records.tail(len(edm_records) / 2)
rap_train     = rap_records.head(len(rap_records) / 2)
rap_test      = rap_records.tail(len(rap_records) / 2)

# Create big training and big test set for analysis
training_set = pd.concat([country_train,rock_train,pop_train,edm_train,rap_train])
test_set     = pd.concat([country_test,rock_test,pop_test,edm_test,rap_test])

training_set = training_set.fillna(0)
test_set = test_set.fillna(0)

print "Training Records:\t" , len(training_set)
print "Test Records:\t\t" , len(test_set)
# training_set.head()

Training Records:	405
Test Records:		405


### Generating Random Forest - Viewer Statistics
We start generating our random forests, and output a relative accuracy and a confusion matrix. In this first one, we simply factor in non-color variables (rating, likes, dislikes, length and viewcount), and run it across all records to predict an ordinal genre value.

In [5]:
# Predicting based solely on non-color features, using RF
clf = RandomForestClassifier(n_estimators=11)
meta_data_features = ['rating', 'likes','dislikes','length','viewcount']
y, _ = pd.factorize(training_set['genre_ordinal'])
clf = clf.fit(training_set[meta_data_features], y)

z, _ = pd.factorize(test_set['genre_ordinal'])
print clf.score(test_set[meta_data_features],z)
pd.crosstab(test_set.genre_ordinal, clf.predict(test_set[meta_data_features]),rownames=["Actual"], colnames=["Predicted"])

0.444444444444


Predicted,0,1,2,3,4
Actual,,,,,
0,51,5,0,21,4
1,2,24,41,7,7
2,32,8,3,29,3
3,8,14,23,29,9
4,7,18,2,7,51


As shown above, this method yields relatively poor results. This is because there's no distinct clusters being created by our random forest, and simple viewer statistics tell us nothing about what kind of video we're watching. However, we see that country, rap and pop are initially somewhat distinct (diagonal is the highest value), and rock and edm are getting mistaken for one another. Let's see if we can't make something of this.

### Random Forest - Only Color Statistics
Below, we do the same random forest as above, but going strictly off of average frame color for the video.

We found the most commonly appearing color in each frame and called it the 'frame mode'. We then took all of the frame modes and found the 10 most common of them. Those became the 'color data' we use to analyze videos.

In [6]:
def gen_new_headers(old_headers):
    headers = ['colors_' + str(x+1) + '_' for x in range(10)]
    h = []
    for x in headers:
        h.append(x + 'red')
        h.append(x + 'blue')
        h.append(x + 'green')
    return old_headers + h + ['genre']

In [7]:
clf = RandomForestClassifier(n_estimators=11)
color_features = gen_new_headers([])[:-1]

# Predicting based solely on colors
y, _ = pd.factorize(training_set['genre_ordinal'])
clf = clf.fit(training_set[color_features], y)

z, _ = pd.factorize(test_set['genre_ordinal'])
print clf.score(test_set[color_features],z)
pd.crosstab(test_set.genre_ordinal, clf.predict(test_set[color_features]),rownames=["Actual"], colnames=["Predicted"])

0.222222222222


Predicted,0,1,2,3,4
Actual,,,,,
0,31,17,16,10,7
1,10,15,20,25,11
2,38,11,5,13,8
3,14,16,30,15,8
4,21,19,21,11,13


This actually yields worse results than just the viewer statistics, because the color of a video by itself does not determine the genre. If rappers only had red in their videos and rockers only had black this might be somewhat accurate, but that's just not the case. But, what if we pair these findings with our initial viewer statistics? 

### Random Forest - All Features

In [8]:
clf = RandomForestClassifier(n_estimators=11)
all_features = meta_data_features + color_features

# Predicting based on colors and non-color features
y, _ = pd.factorize(training_set['genre_ordinal'])
clf = clf.fit(training_set[all_features], y)

z, _ = pd.factorize(test_set['genre_ordinal'])
print clf.score(test_set[all_features],z)
pd.crosstab(test_set.genre_ordinal, clf.predict(test_set[all_features]),rownames=["Actual"], colnames=["Predicted"])

0.372839506173


Predicted,0,1,2,3,4
Actual,,,,,
0,46,9,2,15,9
1,5,24,38,9,5
2,21,18,4,23,9
3,10,17,25,15,16
4,20,17,2,12,34


### Singling Out Pop and Rap
Scores are expectedly low. It seems as if we're trying to make the classifier do way too much work, and are giving it very mediocre data to go off of. Recall that we're actually trying to determine WHICH genre a video is by the above code, not whether or not a video is of ONE specific genre. This brings back the binary classifiers that we created above, let's put those to use to see if we can improve these scores.

We try pop and rap first, since they seem to be the most distinct by what we've gathered above.

In [9]:
clf = RandomForestClassifier(n_estimators=11)
all_features = meta_data_features + color_features

# Predicting based on colors and non-color features
y, _ = pd.factorize(training_set['is_pop'])
clf = clf.fit(training_set[all_features], y)

z, _ = pd.factorize(test_set['is_pop'])
print clf.score(test_set[all_features],z)
pd.crosstab(test_set.is_pop, clf.predict(test_set[all_features]),rownames=["Actual"], colnames=["Predicted"])

0.81975308642


Predicted,0,1
Actual,,
0,311,13
1,60,21


In [10]:
clf = RandomForestClassifier(n_estimators=11)
all_features = meta_data_features + color_features

# Predicting based on colors and non-color features
y, _ = pd.factorize(training_set['is_rap'])
clf = clf.fit(training_set[all_features], y)

z, _ = pd.factorize(test_set['is_rap'])
print clf.score(test_set[all_features],z)
pd.crosstab(test_set.is_rap, clf.predict(test_set[all_features]),rownames=["Actual"], colnames=["Predicted"])

0.775308641975


Predicted,0,1
Actual,,
0,291,29
1,62,23


What we're seeing above is a confusion matrix that, based on our training data, predicts whether or not a video in the test set is a pop video or not. In the "predicted" row, 0 means it predicts it's not a pop video, and that the 1 is. Likewise with the actual, 0 shows that the video actually wasn't a pop video, and the 1 shows that it was.

The confusion matrix above is our first effort at utilizing these binary classifiers. Most of our videos aren't pop videos, and the model did a good job of picking out those that aren't pop. However, we could use some improvement in the realm of "false negatives", where the model classified a video as not pop when it actually was.

We do these tests 50 times for sake of average score.

Rather than hard-coding each time we wanted to run something for average, we wrote a function that does it for us. All we have to do is pass in the boolean classifier in quotes ("is_rock", etc.), and the number of iterations that we want. Results are displayed below.

In [18]:
def multi_RF_averages(is_genre,num_iterations):
    clf = RandomForestClassifier(n_estimators=11)
    loop_indices = range(0,num_iterations)
    cumsum = 0

    for i in loop_indices:
        y, _ = pd.factorize(training_set[is_genre])
        clf = clf.fit(training_set[all_features], y)

        z, _ = pd.factorize(test_set[is_genre])
        cumsum = cumsum + clf.score(test_set[all_features],z)
    
    
    print "Average Score for",len(loop_indices),is_genre,"iterations:", cumsum/len(loop_indices)
    return clf

In [20]:
pop_class = multi_RF_averages("is_pop",50)
rap_class = multi_RF_averages("is_rap",50)
rock_class = multi_RF_averages("is_rock",50)
edm_class = multi_RF_averages("is_edm",50)
country_class = multi_RF_averages("is_country",50)

Average Score for 50 is_pop iterations: 0.904303797468
Average Score for 50 is_rap iterations: 0.889493670886
Average Score for 50 is_rock iterations: 0.588607594937
Average Score for 50 is_edm iterations: 0.51835443038
Average Score for 50 is_country iterations: 0.0744303797468


The following creates several files that describe our classifiers. Our website will later  

In [24]:
from sklearn.externals import joblib
joblib.dump(pop_class, 'classifiers/pop_class.pkl')
joblib.dump(rap_class, 'classifiers/rap_class.pkl')
joblib.dump(rock_class, 'classifiers/rock_class.pkl')
joblib.dump(edm_class, 'classifiers/edm_class.pkl')
joblib.dump(country_class, 'classifiers/country_class.pkl')

['classifiers/counry_class.pkl']

We ran the above test with all genres, and as shown in above analysis, our country and edm typically have very low accuracy. We've seen above that edm and rock videos are getting mixed up with one another, so we assume that something is characteristic of these 2 genres that's not of everything else. We take out the edm values from our training and test datasets, hoping to improve accuracy.

In [13]:
# Removing EDM for better analysis - makes is_pop and is_rap much more accurate
training_set = pd.concat([country_train,rock_train,pop_train,rap_train])
test_set     = pd.concat([country_test,rock_test,pop_test,rap_test])

multi_RF_averages("is_pop",50)
multi_RF_averages("is_rap",50)
multi_RF_averages("is_rock",50)
multi_RF_averages("is_edm",50)
multi_RF_averages("is_country",50)

Average Score for 50 is_pop iterations: 0.841428571429
Average Score for 50 is_rap iterations: 0.748819875776
Average Score for 50 is_rock iterations: 0.768385093168
Average Score for 50 is_edm iterations: 1.0
Average Score for 50 is_country iterations: 0.739254658385


So, what does this tell us? Based on our training data, we have the best chance of accurately classifying something as pop or not pop (under these conditions). 

We want to find out which 2 are the most distinct, so we can make build our model based on that classification.

In [14]:
training_set = pd.concat([country_train,rock_train,edm_train,rap_train,pop_train])

test_set     = pd.concat([rock_test])
multi_RF_averages("is_rock",50)

test_set     = pd.concat([rap_test])
multi_RF_averages("is_rap",50)

test_set     = pd.concat([country_test])
multi_RF_averages("is_country",50)

test_set     = pd.concat([pop_test])
multi_RF_averages("is_pop",50)

test_set     = pd.concat([edm_test])
multi_RF_averages("is_edm",50)

Average Score for 50 is_rock iterations: 0.8104
Average Score for 50 is_rap iterations: 0.736705882353
Average Score for 50 is_country iterations: 0.212345679012
Average Score for 50 is_pop iterations: 0.736049382716
Average Score for 50 is_edm iterations: 0.920240963855


Rock and EDM have suprisingly distinct classifiers. We should dive into the videos and see what this means.

In [15]:
test_set     = pd.concat([edm_test,rock_test])
multi_RF_averages("is_edm",50)
multi_RF_averages("is_rock",50)

Average Score for 50 is_edm iterations: 0.525696202532
Average Score for 50 is_rock iterations: 0.58582278481


### Selecting Most Valuable Features per Genre - Rock

In [16]:
model = ExtraTreesClassifier()

training_set = pd.concat([country_train,pop_train,rap_train,rock_train,edm_train])
y, _ = pd.factorize(training_set['is_rock'])
model.fit(training_set[all_features], y)

# display the relative importance of each attribute
print model.feature_importances_

[ 0.03290921  0.03469756  0.03118444  0.04860914  0.01782878  0.01999346
  0.03024935  0.04399394  0.02505887  0.03233096  0.02809611  0.02450839
  0.03535645  0.02962933  0.01802695  0.02066237  0.03197878  0.02706119
  0.02530101  0.0317604   0.0277708   0.03673455  0.03398566  0.02701643
  0.02198703  0.02476635  0.02466785  0.02749125  0.04315477  0.0293925
  0.01777877  0.02000058  0.02779286  0.02196882  0.02625507]


In [17]:
df = pd.DataFrame()
df['index'] = all_features

y, _ = pd.factorize(training_set['is_rap'])
model.fit(training_set[all_features], y)
        
df['rap'] = model.feature_importances_

y, _ = pd.factorize(training_set['is_rock'])
model.fit(training_set[all_features], y)

df['rock'] = model.feature_importances_

y, _ = pd.factorize(training_set['is_country'])
model.fit(training_set[all_features], y)

df['country'] = model.feature_importances_

y, _ = pd.factorize(training_set['is_edm'])
model.fit(training_set[all_features], y)

df['edm'] = model.feature_importances_

y, _ = pd.factorize(training_set['is_pop'])
model.fit(training_set[all_features], y)

df['pop'] = model.feature_importances_

df

,index,rap,rock,country,edm,pop
0,rating,0.037061,0.040522,0.034231,0.040594,0.037252
1,likes,0.040754,0.034703,0.049799,0.017125,0.188497
2,dislikes,0.034943,0.033566,0.034409,0.023285,0.145784
3,length,0.041574,0.049036,0.024699,0.022436,0.018454
4,viewcount,0.058718,0.028356,0.039757,0.037858,0.108199
5,colors_1_red,0.027543,0.032578,0.023257,0.033716,0.015370
6,colors_1_blue,0.029278,0.030500,0.031848,0.027285,0.018871
7,colors_1_green,0.023380,0.030655,0.023208,0.028503,0.016904
8,colors_2_red,0.030502,0.028434,0.025461,0.023928,0.010516
9,colors_2_blue,0.023691,0.025346,0.029427,0.027073,0.021818


### Future Improvements 
- Run the above graph a number of times, take the average for each cell
- Based on the heaviest weighted parameters for each, run the random forest algorithm only taking these given parameters into consideration
- Generate a model that classifies videos dynamically
- Make more values ordinal - maybe to NLP or LDA to factor in descriptions, titles and lyrics